In [1]:
import numpy as np
import pandas as pd
import logging
import os
from datetime import timedelta
from pathlib import Path

from tinkoff.invest import CandleInterval, Client
from tinkoff.invest.utils import now
from tinkoff.invest.caching.market_data_cache.cache import MarketDataCache
from tinkoff.invest.caching.market_data_cache.cache_settings import (
    MarketDataCacheSettings,
)

from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

import sys
sys.path.append("..") 
from Portfolio.portfolio_tools import *

import tink_port as tink

Вводим токен, получаем список доступных аккаунтов

In [2]:
BASE = 't.UFRJ8SC9hafVOhFxEUY7yf1wZ1gGhwJp-WCp9o4rnEChHWns0c3jQ21eQwoOW_RurFqeZpss2scJkmMQnomJ9g'
ETF = 't.6nHltT1dYSfrVTIV9zF72fxDlB2sXJbRD6iJNpZXTFAN61rmD7m71xPp9ko12ta1JxA06em4YdN36xicnBmjWg'
MOMENTUM = 't.24WV5_MMG1bQArK1WPp1_DYWD52f-VfGjpR1ci5Pqf0PJ948zWhDstoO_6d4wXIhFTMVsVJSgOzPElUIPEO4Mw'

token = MOMENTUM

In [50]:
accs = tink.get_accounts(token)
print("Количество аккаунтов:", len(accs.accounts))

print(accs.accounts[0].name)
account_id = accs.accounts[0].id

Количество аккаунтов: 1
Momentum 90 Days


#### База идентификаторов Tinkoff

In [4]:
base = tink.get_id_base(token)
port = tink.get_portfolio(token)
df_port = tink.port_to_df(port, base)

In [5]:
tink.get_asset_lot('BBG004S68614', base)

100

In [6]:
df_port

,figi,ticker,name,quantity,lot_quantity,price
3,BBG004S68614,AFKS,АФК Система,1900,19,24.07
1,BBG004S68758,BANE,Башнефть,4,4,3166.50
8,BBG00475K6C3,CHMF,Северсталь,4,4,1859.00
2,BBG00Y91R9T3,OZON,Ozon Holdings PLC,1,1,3801.00
6,TCS00A103X66,POSI,Positive Technologies,2,2,2886.00
0,BBG004S682Z6,RTKM,Ростелеком,80,8,100.50
7,BBG004S685M3,RTKMP,Ростелеком - Привилегированные акции,270,27,85.35
5,TCS00A105EX7,WUSH,Whoosh,20,20,320.87
4,RUB000UTSTOM,None,None,3133,3133,1.00


### Portfolio Black-Litterman Model

In [8]:
mcaps = {}
for t in dfp.columns:
   # mcaps[t] = stock.info["marketCap"]
    mcaps[t] = 1
mcaps

{'LKOH': 1,
 'NLMK': 1,
 'AGRO': 1,
 'MAGN': 1,
 'TATN': 1,
 'SIBN': 1,
 'BANEP': 1,
 'SNGSP': 1,
 'POSI': 1,
 'SBER': 1,
 'CHMF': 1,
 'ROSN': 1}

In [9]:
mcaps['SBER'] = 1.5
mcaps['POSI'] = 1.5

In [28]:
imoex_figi = 'BBG333333333'
market_prices = tink.get_candles(token, imoex_figi, CandleInterval.CANDLE_INTERVAL_DAY)
market_prices =  tink.get_open_price(market_prices)

In [21]:
dfp

,LKOH,NLMK,AGRO,MAGN,TATN,SIBN,BANEP,SNGSP,POSI,SBER,CHMF,ROSN
date,,,,,,,,,,,,
2023-10-02,6716.5,210.50,1208.0,52.650,628.0,722.10,1468.0,58.500,2372.2,261.37,1371.0,538.50
2023-10-03,6720.0,209.96,1190.0,52.955,639.0,711.00,1439.5,56.800,2420.0,258.99,1392.0,534.65
2023-10-04,6740.0,208.86,1179.8,51.500,640.0,723.90,1432.5,56.650,2397.0,259.00,1355.6,538.20
2023-10-05,6706.0,212.84,1149.8,52.250,634.2,704.45,1475.0,55.475,2440.0,260.70,1370.0,536.00
2023-10-06,6749.5,210.80,1158.8,51.995,635.0,718.10,1460.0,57.090,2435.0,259.77,1369.4,534.50
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-21,7450.0,212.78,1411.8,54.800,695.7,755.70,2142.0,61.845,2674.0,295.99,1743.2,561.30
2024-03-22,7420.0,217.98,1422.6,55.520,704.1,756.80,2143.0,62.490,2680.0,295.77,1796.6,562.85
2024-03-25,7305.0,214.50,1388.0,54.800,694.5,737.00,2120.0,61.945,2642.2,294.00,1777.8,556.70


In [38]:
from pypfopt import black_litterman, risk_models
from pypfopt import BlackLittermanModel

S = risk_models.CovarianceShrinkage(dfp).ledoit_wolf()
delta = black_litterman.market_implied_risk_aversion(market_prices['ticker'])
delta

9.802403162030824

In [39]:
market_prior = black_litterman.market_implied_prior_returns(mcaps, delta, S)
market_prior

LKOH     0.235540
NLMK     0.250621
AGRO     0.404671
MAGN     0.222163
TATN     0.175982
SIBN     0.177426
BANEP    0.306354
SNGSP    0.265854
POSI     0.254061
SBER     0.201803
CHMF     0.241332
ROSN     0.239506
dtype: float64

In [41]:
# You don't have to provide views on all the assets
viewdict = {
    "POSI": 0.10,
}

confidences = [
    0.6
]
bl = BlackLittermanModel(S, pi=market_prior, absolute_views=viewdict)

In [43]:
intervals = [
    (0.1, 0.4),
]

variances = []
for lb, ub in intervals:
    sigma = (ub - lb)/2
    variances.append(sigma ** 2)

print(variances)
omega = np.diag(variances)

[0.022500000000000006]


In [44]:
# We are using the shortcut to automatically compute market-implied prior
bl = BlackLittermanModel(S, pi="market", market_caps=mcaps, risk_aversion=delta,
                        absolute_views=viewdict, omega=omega)

In [45]:
# Posterior estimate of returns
ret_bl = bl.bl_returns()
ret_bl

LKOH     0.210907
NLMK     0.225580
AGRO     0.377385
MAGN     0.198196
TATN     0.153239
SIBN     0.155086
BANEP    0.281036
SNGSP    0.241803
POSI     0.213216
SBER     0.178689
CHMF     0.218008
ROSN     0.216051
dtype: float64

In [46]:
rets_df = pd.DataFrame([market_prior, ret_bl, pd.Series(viewdict)], 
             index=["Prior", "Posterior", "Views"]).T
rets_df

,Prior,Posterior,Views
LKOH,0.235540,0.210907,NaN
NLMK,0.250621,0.225580,NaN
AGRO,0.404671,0.377385,NaN
MAGN,0.222163,0.198196,NaN
TATN,0.175982,0.153239,NaN
SIBN,0.177426,0.155086,NaN
BANEP,0.306354,0.281036,NaN
SNGSP,0.265854,0.241803,NaN
POSI,0.254061,0.213216,0.1
SBER,0.201803,0.178689,NaN


In [49]:
S_bl = bl.bl_cov()


In [50]:
from pypfopt import EfficientFrontier, objective_functions

ef = EfficientFrontier(ret_bl, S_bl)
ef.add_objective(objective_functions.L2_reg)
ef.max_sharpe()
weights = ef.clean_weights()
weights
     

D:\Soft\Anaconda\envs\t\Lib\site-packages\pypfopt\efficient_frontier\efficient_frontier.py:259: UserWarning: max_sharpe transforms the optimization problem so additional objectives may not work as expected.
  warnings.warn(


OrderedDict([('LKOH', 0.07897),
             ('NLMK', 0.08567),
             ('AGRO', 0.14173),
             ('MAGN', 0.07393),
             ('TATN', 0.05579),
             ('SIBN', 0.0562),
             ('BANEP', 0.10686),
             ('SNGSP', 0.09078),
             ('POSI', 0.08075),
             ('SBER', 0.06611),
             ('CHMF', 0.08197),
             ('ROSN', 0.08124)])

In [52]:
from pypfopt import DiscreteAllocation

da = DiscreteAllocation(weights, dfp.iloc[-1], total_portfolio_value=100000)
alloc, leftover = da.lp_portfolio()
print(f"Leftover: ${leftover:.2f}")
alloc

Leftover: $703.29


D:\Soft\Anaconda\envs\t\Lib\site-packages\cvxpy\problems\problem.py:1403: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


{'LKOH': 1,
 'NLMK': 39,
 'AGRO': 10,
 'MAGN': 130,
 'TATN': 8,
 'SIBN': 8,
 'BANEP': 5,
 'SNGSP': 141,
 'POSI': 3,
 'SBER': 22,
 'CHMF': 4,
 'ROSN': 15}

### Model

## Load all shares

In [7]:
dfx = base[base["type"] == "shares"]
dfx = dfx[dfx["cur"] == "rub"]
base_ru = dfx.copy()

In [8]:
res = []
for ind, pos in base_ru.iterrows():
#    print(pos.figi)        
    candles = tink.get_candles(token, pos.figi, CandleInterval.CANDLE_INTERVAL_DAY, 50)
    df =  tink.get_open_price(candles)
    ticker = tink.figi_to_ticker(pos.figi, base)
    
    if ticker == None:
        ticker = pos.figi
    df.columns = [ticker]
    res.append(df)
    
df_full = pd.concat(res, axis = 1)

#### Drops Momentum

In [9]:
dfp = df_full.copy()
drops = ["GTRK", "NTZL", "LSRG", 'TGKBP', 'FIVE', "VEON-RX"]
columns = [x for x in dfp.columns if x not in drops]
dfp = dfp[columns]


#### Drops Base

In [17]:
dfp = df_full.copy()
index_assets = pd.read_csv('index_assets.csv')['asset'].values.tolist()
columns = [x for x in dfp.columns if x in index_assets]

drops = ['FIVE']
columns = [x for x in dfp.columns if x not in drops]

dfp = dfp[columns]

### RiskFolio

In [10]:
import riskfolio as rp

rms= ['MV', 'MSV',  'CVaR']

lookback = 30
df_period = dfp[-lookback:]

def riskfolio_weights(df_period, rm , obj):
    """
        obj - Objective function, could be MinRisk, MaxRet, Utility or Sharpe
    """
    Y = df_period.pct_change().dropna()

    # Building the portfolio object
    port = rp.Portfolio(returns=Y)
    port.solvers = ['MOSEK']
    # Calculating optimum portfolio

    # Select method and estimate input parameters:

    method_mu='hist' # Method to estimate expected returns based on historical data.
    method_cov='hist' # Method to estimate covariance matrix based on historical data.

    port.assets_stats(method_mu=method_mu, method_cov=method_cov, d=0.94)

    # Estimate optimal portfolio:

    model='Classic' # Could be Classic (historical), BL (Black Litterman) or FM (Factor Model)
    hist = True # Use historical scenarios for risk measures that depend on scenarios
    rf = 0 # Risk free rate
    l = 0 # Risk aversion factor, only useful when obj is 'Utility'
    # First we need to delete the cardinality constraint
    port.card = None 

    # Then we need to set the constraint on the minimum number of effective assets
    port.nea = 10
    w = port.optimization(model=model, rm=rm, obj=obj, rf=rf, l=l, hist=hist)
    w = w[w.weights > 0.01]
    return w

#print_data(dfp)
df_period = dfp.dropna(axis = 1)

dfw = riskfolio_weights(df_period, 'CVaR', 'MaxRet')
dfw

You must convert self.cov to a positive definite matrix


,weights
WUSH,0.020454915
UGLD,0.052492146
HHRU,0.011809049
SELG,0.012389420
RTKM,0.050751753
GCHE,0.020867767
OZON,0.036326022
GLTR,0.030307263
RTKMP,0.029445959
AFLT,0.018677611


In [25]:
dfx = final_sums(dfw, 116000, 50)
dfx['lot'] = 1
inds = dfx.index.values.tolist()

x = base_ru[base_ru['ticker'].isin (inds)]
s = x[['ticker', 'lot']].set_index('ticker')
dfx['lot'] = s

In [26]:
prices = dfp.iloc[-1].T.loc[dfx.index]
dfx["price"] = prices
dfx["lot_quantity"] = np.round(dfx.weights/ (dfx.price * dfx.lot)).astype(int)
dfx["quantity"] =  dfx.lot * dfx.lot_quantity
dfx["sum"]= dfx.price * dfx.lot_quantity * dfx.lot
dfx = dfx.sort_values("weights", ascending = False)
dfx.to_csv("t.csv")
dfx = dfx.reset_index()
dfx.columns = ['ticker', 'weights', 'lot', 'price', 'lot_quantity','quantity', 'sum']

dfx

,ticker,weights,lot,price,lot_quantity,quantity,sum
0,SFIN,23930.0,1,1820.000,13,13,23660.00
1,RBCM,21060.0,100,17.500,12,1200,21000.00
2,RNFT,10380.0,1,237.500,44,44,10450.00
3,UGLD,6090.0,1000,0.960,6,6000,5760.00
4,RTKM,5890.0,10,100.950,6,60,6057.00
5,AFKS,5040.0,100,23.454,2,200,4690.80
6,POSI,5020.0,1,2943.600,2,2,5887.20
7,OZON,4210.0,1,3845.500,1,1,3845.50
8,GLTR,3520.0,1,815.000,4,4,3260.00
9,RTKMP,3420.0,10,85.000,4,40,3400.00


In [27]:
df_port

,figi,ticker,name,quantity,lot_quantity,price
3,BBG004S68614,AFKS,АФК Система,1900,19,24.07
1,BBG004S68758,BANE,Башнефть,4,4,3166.50
8,BBG00475K6C3,CHMF,Северсталь,4,4,1859.00
2,BBG00Y91R9T3,OZON,Ozon Holdings PLC,1,1,3801.00
6,TCS00A103X66,POSI,Positive Technologies,2,2,2886.00
0,BBG004S682Z6,RTKM,Ростелеком,80,8,100.50
7,BBG004S685M3,RTKMP,Ростелеком - Привилегированные акции,270,27,85.35
5,TCS00A105EX7,WUSH,Whoosh,20,20,320.87
4,RUB000UTSTOM,None,None,3133,3133,1.00


In [32]:
def df_to_dict(dfx):
    return {row.ticker:row.lot_quantity for _,row in dfx.iterrows()}

old_port = df_to_dict(df_port)
new_port = df_to_dict(dfx)
new_port

{'SFIN': 13,
 'RBCM': 12,
 'RNFT': 44,
 'UGLD': 6,
 'RTKM': 6,
 'AFKS': 2,
 'POSI': 2,
 'OZON': 1,
 'GLTR': 4,
 'RTKMP': 4,
 'DELI': 10,
 'CIAN': 3,
 'ASTR': 4,
 'GCHE': 0,
 'WUSH': 8,
 'AFLT': 5,
 'LENT': 2,
 'SELG': 2,
 'HHRU': 0,
 'PLZL': 0}

In [33]:
old_port

{'AFKS': 19,
 'BANE': 4,
 'CHMF': 4,
 'OZON': 1,
 'POSI': 2,
 'RTKM': 8,
 'RTKMP': 27,
 'WUSH': 20,
 None: 3133}

In [34]:
def calculate_portfolio_difference(old_portfolio, new_portfolio):
    """
    Рассчитать разницу между двумя портфелями.

    Args:
        old_portfolio: Словарь, где ключом является тикер, а значением - количество акций.
        new_portfolio: Словарь, где ключом является тикер, а значением - количество акций.

    Returns:
        Словарь, где ключом является тикер, а значением - разница между количеством акций в 
        новых и старых портфелях.
    """

    difference = {}
    for ticker in new_portfolio:
        if ticker in old_portfolio:
            difference[ticker] = new_portfolio[ticker] - old_portfolio[ticker]
        else:
            difference[ticker] = new_portfolio[ticker]

    for ticker in old_portfolio:
        if ticker not in new_portfolio:
            difference[ticker] = -old_portfolio[ticker]
    # Сортировка по значению, по возрастанию
    sorted_diff = sorted(difference.items(), key=lambda x: x[1])

    return sorted_diff

In [36]:
rebalance = calculate_portfolio_difference(old_port, new_port)

flag_rebalance = True

In [57]:
for asset, qty in rebalance[12:]:
    if asset is None:
        continue
        
    print(asset, qty)

LENT 2
SELG 2
CIAN 3
GLTR 4
ASTR 4
AFLT 5
UGLD 6
DELI 10
RBCM 12
SFIN 13
RNFT 44


In [58]:
from tinkoff.invest import OrderDirection, OrderType
residuals = []
if flag_rebalance:
    with Client(token) as client:

        for asset, qty in rebalance[12:]:
            print(asset)
            if asset is None:
                continue
                
            figi = tink.ticker_to_figi(asset, base)
            trading_status = client.market_data.get_trading_status(
                figi=figi
            )
            
            if trading_status.market_order_available_flag and trading_status.api_trade_available_flag:
                if qty < 0:
                    resp = client.orders.post_order(figi=figi,
                                quantity= -qty,
                                direction=OrderDirection.ORDER_DIRECTION_SELL,
                                account_id=account_id,
                                order_type=OrderType.ORDER_TYPE_MARKET,)
                elif qty > 0:
                    resp = client.orders.post_order(figi=figi,
                        quantity=qty,
                        direction=OrderDirection.ORDER_DIRECTION_BUY,
                        account_id=account_id,
                        order_type=OrderType.ORDER_TYPE_MARKET,)
            else:
                print("Не доступно")
                residuals.append((asset, qty))
    flag_rebalance = False

LENT
Не доступно
SELG
CIAN
Не доступно
GLTR
ASTR
AFLT
UGLD
Не доступно
DELI
Не доступно
RBCM
Не доступно
SFIN
Не доступно
RNFT


In [59]:
residuals

[('LENT', 2),
 ('CIAN', 3),
 ('UGLD', 6),
 ('DELI', 10),
 ('RBCM', 12),
 ('SFIN', 13)]